# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [1]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

--2020-12-11 13:25:34--  https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 198.35.26.112, 2620:0:863:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|198.35.26.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223725 (218K) [image/jpeg]
Saving to: ‘La_noche_estrellada1.jpg’

La_noche_estrellada 100%[===================>] 218.48K  --.-KB/s    in 0.08s   

2020-12-11 13:25:34 (2.80 MB/s) - ‘La_noche_estrellada1.jpg’ saved [223725/223725]

--2020-12-11 13:25:34--  https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 198.35.26.112, 2620:0:863:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|198.35.26.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153015 (149K) [image/j

In [2]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [82]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

#base_image_path = Path("/content/775px-Neckarfront_Tübingen_Mai_2017.jpg")
#style_reference_image_path = Path("/content/La_noche_estrellada1.jpg")
base_image_path = Path("/content/Obelisco.jpg")
style_reference_image_path = Path("/content/berni_desocupados.jpg")

result_prefix = Path("/content/output")
iterations = 100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:


1) total_variation_weight = Es el peso de la loss que regula los componentes de alta frecuencia de la imagen (rugosidad, ruido)

2) style_weight = Es el peso que le vamos a dar a la reconstrucción de estilo. Se multiplica por la style loss 

3) content_weight = Es el peso que le vamos a dar a la reconstrucción de contenido. Se multiplica por la content loss

In [83]:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

In [84]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta:

 

1)   load_img carga la imagen y cambia el tamaño al especificado en tarjet_size (400 filas x 517 columnas) 

2)   img_to_array() convierte la imagen cargada en formato PIL a NumPy array de dimensión (400,517,3). El 3 viene de RGB, 3 canales


3)   np.expand_dims agrega una dimensión para poder agregar el número de muestras, ahora la dimensión de la imagen es (1, 400, 517, 3)

4)   vgg19.preprocess_input(img) se encarga de preprocesar la imagen y adecuarla al formato (normalizado, etc) requerido por el modelo, en este caso vgg19












 

 

 

In [85]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta:

Esta función procesa la imagen para guardarla. Primero vuelve a transformar la imagen a 3 dimesiones y luego la pasa de BGR a RGB invirtiendo el orden de los canales (3a dimensión). *Finalmente* convierte los valores <0 a 0 y los > 255 a 255

In [8]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [86]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [87]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [88]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [89]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

Model loaded.


# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?
- ¿Por qué se permutan las dimensiones de x?

1) La matriz de Gram es el producto punto entre todos los features, que no es otra cosa que la correlación entre los features (a mayor producto punto entre dos vectores mayor correlación). Es una representación del estilo de la imagen, por lo tanto se utiliza en la style loss para medir como difiere el estilo de la imagen generada de la style image de entrada comparando las dos matrices de Gram respectivas.

2) Las dimensiones de x se permutan porque la matrix de Gram debe tener dimensión = NXN donde N es la cantidad de filtros o canales. Gram = M.M_transpuesta donde dim(M) = NxP y P = ancho x alto del feature. Por lo tanto hay que poner a los canales como primera dimensión de la Matriz x.

In [13]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:

Style loss es la loss function que representa a la imágen de estilo. Es una medida de la similitud entre la imágen de salida y la "style image" de entrada

Content loss es la loss function que representa a la imágen de contenido. Es una medida de la similitud entre la imágen de salida y la "content image" de entrada

Total variation loss es la suma de las diferencias absolutas entre pixeles contiguos de la imágen de entrada. Es una medida de cuanto ruido tiene la imágen. Al agregarla a la loss de training se remueve la rugosidad en la textura de la imágen y como resultado obtenemos una imágen suavizada.

In [14]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [15]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [16]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [90]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [91]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:

1) eval_loss_and_grads devuelve el valor de la loss y del gradiente en cada iteración

2) La clase Evaluator devuelve la loss y el gradiente en dos funciones separadas debido a una requerimiento del optimizador, de todos modos para que sea eficiente se computan los dos dentro de la función "loss"

3) la 3 celda itera optimizando la loss en cada paso y guarda la imagen generada

4)fmin_l_bfgs_b es el optimizador utilizado, devuelve la imagen transformada que minimiza la loss

5) Se diferencian en que en el paper utilizan como optimizador gradient descent

6) Se podría usar ADAM

In [19]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [20]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [92]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 52150456000.0
Image saved as /content/output/output_at_iteration_0.png
Iteration 0 completed in 8s
Start of iteration 1
Current loss value: 22480300000.0
Image saved as /content/output/output_at_iteration_1.png
Iteration 1 completed in 6s
Start of iteration 2
Current loss value: 15200647000.0
Image saved as /content/output/output_at_iteration_2.png
Iteration 2 completed in 6s
Start of iteration 3
Current loss value: 12224016000.0
Image saved as /content/output/output_at_iteration_3.png
Iteration 3 completed in 6s
Start of iteration 4
Current loss value: 10369350000.0
Image saved as /content/output/output_at_iteration_4.png
Iteration 4 completed in 6s
Start of iteration 5
Current loss value: 9267375000.0
Image saved as /content/output/output_at_iteration_5.png
Iteration 5 completed in 6s
Start of iteration 6
Current loss value: 8469459000.0
Image saved as /content/output/output_at_iteration_6.png
Iteration 6 completed in 6s
Start of iteration 7
C

In [ ]:
!zip -r /content/images.zip /content/output

In [94]:
from google.colab import files
files.download("/content/images.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:

Las imágenes generadas en cada corrida están en las siguientes carpetas :

outputs_1 : Imágenes generadas con style=10 y content=1, se le da más importancia al estilo, la imagen generada capta muy bien el estilo.

outputs_2 :Imágenes generadas con style=1 y content=500, se le da más importancia al contenido. La imagen generada reproduce muy bien el contenido pero no tan bien el estilo.

outputs_3 :Imágenes generadas con style=500 y content=1, se le da un excesivo peso al estilo y no logra representarse bien el contenido.


# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta:

outputs_4 :Imágenes propias generadas con style=10 y content=1.

input contenido : Obelisco.jpg

imput style : berni_desocupados.jpg